In [30]:
try :
    import gymnasium_robotics
    !echo 'gymnasium-robotics installed'
except :
    print('gymnasium-robotics not installed')
    !pip install gymnasium-robotics

gymnasium-robotics installed


In [1]:
#@title Check if installation was successful

#from google.colab import files

import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

Mon Nov 20 02:52:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   40C    P8    22W / 175W |     94MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

/tmp/ipykernel_1732052/3038417431.py:5: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.util


In [28]:
import random
import gymnasium as gym
import cv2

# https://robotics.farama.org/envs/maze/point_maze/

example_map = [[1, 1, 1, 1, 1, 1, 1, 1],
               [1, 0, 0, 0, 0, 0, 0, 1],
                [1, 0, 0, 0, 0, 0, 0, 1],

       [1, 0, 0, 0, 0, 0, 'r', 1],
        [1, 'c', 0, 0, 0, 0, 0, 1],
               [1, 0, 0, 0, 0, 0, 0, 1],

       [1, 1, 1, 1, 1, 1, 1, 1]]


#env = gym.make('PointMaze_UMaze-v3', maze_map=example_map, render_mode='rgb_array', reward_type='dense')
env = gym.make('PointMaze_UMaze-v3', render_mode='rgb_array', reward_type='dense')

env.reset()

n_frames = 120
height = 480
width = 480
frames = []
#import mujoco


#joint_names = ['thigh_joint', 'leg_joint', 'foot_joint']
#joint_ranges = [env.model.joint(joint).range for joint in joint_names]
for i in range(n_frames):
  new_action = env.action_space
  # get random action Box(-1.0, 1.0, (2,), float32)
  new_action = env.action_space.sample()
  obs, reward, terminated, _, _ = env.step(new_action)
  frame = env.render()
  # add reward to frame
  frame = np.array(frame)
  frame = cv2.putText(frame, f"reward: {reward}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
  #print(obs)
  
  frames.append(frame)
media.show_video(frames, fps=30)


This browser does not support the video tag.
